<a href="https://colab.research.google.com/github/farieu/data-analysis/blob/OutrosModelos/RandomForest_(sem_pipeline).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Treinamento com Random Forest

Esse treinamento é uma nova versão do que estava commitado. A primeira versão do pipeline apresentava vários bugs, e ocasionava na elaboração de um dataset gigantesco, muito diferente do que o que usei para Regressão.

---
Essa nova versão utiliza o GoodReads Encoded, fruto da branch **SelecaoFeatures**, mesmo dataset que usei para treinar o modelo de Regressão.

### Importação de bibliotecas e dataset

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import Binarizer, LabelEncoder, MultiLabelBinarizer
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, ConfusionMatrixDisplay

In [3]:
df = pd.read_csv('/content/drive/MyDrive/BackEnd/Treinamento/GoodReadsEncoded.csv')

### Treinamento sem passar pelo Pipeline de Pré-Processamento

Com o dataset codificado, começo a preparar o treinamento:
dividindo o conjunto em características (features), através

1.   Dividindo o conjunto em X e y
2.  **X**: características/variáveis independentes, que vão servir como entrada para o modelo.
3. **Y**: variável dependente que o modelo está tentando prever, ou seja, a nota do livro.




In [4]:
X = df.drop(columns=['rating', 'desc'])
y = df['rating']

In [5]:
print(X.columns)
print(X.head(2))

Index(['author', 'bookformat', 'pages', 'reviews', 'title', 'totalratings',
       '10th Century', '11th Century', '12th Century', '13th Century',
       ...
       'Young Adult Paranormal', 'Young Adult Romance',
       'Young Adult Science Fiction', 'Young Readers', 'Yuri', 'Zambia', 'Zen',
       'Zimbabwe', 'Zombies', 'æ¼«ç”»'],
      dtype='object', length=1185)
   author  bookformat  pages  reviews  title  totalratings  10th Century  \
0   32032          56      0        5   8698            33             0   
1    8446         100    576        6  21651            41             0   

   11th Century  12th Century  13th Century  ...  Young Adult Paranormal  \
0             0             0             0  ...                       0   
1             0             0             0  ...                       0   

   Young Adult Romance  Young Adult Science Fiction  Young Readers  Yuri  \
0                    0                            0              0     0   
1                   

In [6]:
print(y)
print(y.head(3))

0        3.52
1        4.51
2        4.15
3        3.83
4        3.73
         ... 
84049    3.77
84050    3.97
84051    4.27
84052    3.63
84053    3.83
Name: rating, Length: 84054, dtype: float64
0    3.52
1    4.51
2    4.15
Name: rating, dtype: float64


Conforme mostrado na Preparação do Modelo, precisamos binarizar as notas para treinamento.



*   Avaliações de livros maior ou igual a 4 serão categorizadas como '1' (boa avaliação).
*   Avaliações menores que 4 serão categorizadas como '0' (má avaliação)



In [7]:
binarizer = Binarizer(threshold=3.5)
y_binary = binarizer.fit_transform(y.values.reshape(-1, 1)).ravel()
print(y_binary)

[1. 1. 1. ... 1. 1. 1.]


Agora, precisamos separar os conjuntos de treino e de teste. Para a **Random Forest**, será divido em:

1.   **80% para Treinamento**
2.   **20% para Teste**

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y_binary, test_size=0.2, random_state=42)

Instanciamento do Random Forest, treinamento e avaliação do modelo.

In [9]:
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)

rf_classifier.fit(X_train, y_train)

y_pred = rf_classifier.predict(X_test)

# Avaliar o desempenho do modelo, relatório de classificação e matriz de confusão.
accuracy = accuracy_score(y_test, y_pred)
print(f"Acurácia do modelo: {accuracy:.2f}")
print("\nRelatório de Classificação:\n", classification_report(y_test, y_pred))
print("\nMatriz de Confusão:\n", confusion_matrix(y_test, y_pred))

Acurácia do modelo: 0.88

Relatório de Classificação:
               precision    recall  f1-score   support

         0.0       0.63      0.12      0.21      2043
         1.0       0.89      0.99      0.94     14768

    accuracy                           0.88     16811
   macro avg       0.76      0.56      0.57     16811
weighted avg       0.86      0.88      0.85     16811


Matriz de Confusão:
 [[  251  1792]
 [  148 14620]]


O resultado do modelo Random Forest, quando comparado ao modelo de [Regressão Linear](https://github.com/farieu/data-analysis/blob/AvaliacaoModeloRL/AvaliacaoModeloRL.ipynb), apresenta resultados interessantes.


---

De forma geral, o modelo apresenta alta acurácia, de 88%, mas as métricas para cada classe mostram um problema significativo na identificação de livros com má avaliação (classe 0).

O recall da classe 0 é MUITO baixo, indicando que o modelo não consegue identificar a maioria de má avaliação, classificando-os erroneamente como "boa avaliação". Esse comportamento é evidênciado pela quantidade alta de **falsos positivos (1967)**, o que mostra um viés do modelo em favor da classe 1.


Entretanto, para a classe de boa avaliação, o modelo consegue identificar corretamente todas as instâncias, sem deixar nenhum livro bem avaliado passar despercebido. Isso também é perceptível pelo F1-Score de 94%.

---

O modelo demonstrou similaridade com o **Treinamento 4** de Regressão Logística, o que sugere um possível desbalanceamento das classes, sendo necessário ajustes futuros no modelo.